In [1]:
with open("test.txt") as file:
    instructions = [line.split()[2].strip('()#') for line in file.readlines()]
instructions

['70c710',
 '0dc571',
 '5713f0',
 'd2c081',
 '59c680',
 '411b91',
 '8ceee2',
 'caa173',
 '1b58a2',
 'caa171',
 '7807d2',
 'a77fa3',
 '015232',
 '7a21e3']

In [2]:
charhexmap = {'A':10, 'B':11, 'C':12, 'D':13, 'E':14, 'F':15}
def get_hex_int(hex_char):
    if hex_char.isnumeric():
        return int(hex_char)
    else:
        return charhexmap[hex_char.upper()]

def get_hex_number(hexstr):
    nombor = 0
    for idx,c in enumerate(hexstr[::-1]):
        nombor += get_hex_int(c) * 16 ** idx
    return nombor

hexdirmap = {'0':'R', '1':'D', '2':'L', '3':'U'}

def decode_instr(hex_code):
    digit = get_hex_number(hex_code[:-1])
    dir = hexdirmap[hex_code[-1]]
    return digit, dir

instructions = [decode_instr(instr) for instr in instructions]
instructions

[(461937, 'R'),
 (56407, 'D'),
 (356671, 'R'),
 (863240, 'D'),
 (367720, 'R'),
 (266681, 'D'),
 (577262, 'L'),
 (829975, 'U'),
 (112010, 'L'),
 (829975, 'D'),
 (491645, 'L'),
 (686074, 'U'),
 (5411, 'L'),
 (500254, 'U')]

In [3]:
dir_map = {
    'R':(0, 1),
    'L':(0, -1),
    'D':(1, 0),
    'U':(-1, 0)
}

def get_edge_tile(instr):
    x, y = 0, 0
    edge_tiles = []
    minx = 0
    miny=0
    maxx = 0
    maxy=0
    for length, dir in instr:
        u, v = dir_map[dir]
        nx, ny = x+(u*length), y+(v*length)
        edge_tiles.append((range(x, nx+1), range(y, ny+1)))

        x, y = nx, ny
        minx = minx if minx<x else x
        miny = miny if miny<y else y
        maxx = maxx if maxx>x else x
        maxy = maxy if maxy>y else y

    return edge_tiles, minx, miny, maxx, maxy

tiled, xmin, ymin, xmax, ymax = get_edge_tile(instructions)
W = ymax+1
H = xmax+1
tiled = [(range(x.start-xmin, x.stop-xmin), range(y.start-ymin, y.stop-ymin)) for x, y in tiled]
tiled

[(range(0, 1), range(0, 461938)),
 (range(0, 56408), range(461937, 461938)),
 (range(56407, 56408), range(461937, 818609)),
 (range(56407, 919648), range(818608, 818609)),
 (range(919647, 919648), range(818608, 1186329)),
 (range(919647, 1186329), range(1186328, 1186329)),
 (range(1186328, 1186329), range(1186328, 609067)),
 (range(1186328, 356354), range(609066, 609067)),
 (range(356353, 356354), range(609066, 497057)),
 (range(356353, 1186329), range(497056, 497057)),
 (range(1186328, 1186329), range(497056, 5412)),
 (range(1186328, 500255), range(5411, 5412)),
 (range(500254, 500255), range(5411, 1)),
 (range(500254, 1), range(0, 1))]

In [4]:
def is_intiled(coord):
    a, b = coord
    for xran, yran in tiled:
        if a in xran and b in yran:
            return True
        return False
    
def all_borders():
    upper_border = [(l,e) for e in range(W) for l in (0, H-1)]
    lower_border = [(e, l) for e in range(H) for l in (0, W-1)]
    return [(a,b) for a,b in set(upper_border+lower_border) if not is_intiled((a,b))]

def get_all_valid_neighbord(x, y):
    def inbound(x,y):
        return x in range(H) and y in range(W)
    r = []
    for u,v in ((0,1),(1,0),(0,-1),(-1, 0)):
        nx = x+u
        ny = y+v
        if inbound(nx, ny) and not is_intiled((nx, ny)):
            r.append((nx, ny))
    return r


is_near_border = list(all_borders())


In [5]:
len(is_near_border)

4283374

In [6]:
from tqdm import tqdm
extern_banger = set()
for x, y in tqdm(is_near_border):
    if (x,y) not in extern_banger:
        extern_banger.add((x,y))
        neighbs = [neigb for neigb in get_all_valid_neighbord(x, y) if neigb not in extern_banger]
        is_near_border += neighbs

compte_inte=0
for x in range(H):
    for y in range(W):
        if (x,y) not in extern_banger:
            compte_inte+=1
compte_inte

64191660it [02:13, 317178.15it/s]                            